<a href="https://colab.research.google.com/github/TitapornEve/twitter_Crawler/blob/main/twitterCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pythainlp
!pip install emoji

     |████████████████████████████████| 10.6MB 7.0MB/s 
     |████████████████████████████████| 747kB 33.7MB/s 
     |████████████████████████████████| 133kB 5.2MB/s 


In [61]:
import tweepy
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import emoji
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt 

In [63]:
consumer_key =  '2AVhE9ns3kfaiZAaCSuClF6bw'
consumer_secret = 'IGN07yckRme03ZoZn788gkg8Yjt6PvKgu7BzSPsxWyPHkxxxdA'
access_token = '1362120123339837440-YhXgRzhwnSdnF4E1veeQaxQIyEjbcp'
access_token_secret = 'CRQhrKTHAAKOnBCVae6p9cjcUBjcaJiZMQz4YCFPB3pu3'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
 
api = tweepy.API(auth)

query = "#เด็กคลองหก" 

df = pd.DataFrame(columns=["create_at","text","hashtag","retweet_count","favourite_count"])  
for tweet in tweepy.Cursor(api.search,q=query,count=100,result_type="recent",tweet_mode='extended').items():
    entity_hashtag = tweet.entities.get('hashtags')
    hashtag = ""
    for i in range(0,len(entity_hashtag)):
        hashtag = hashtag +"/"+entity_hashtag[i]["text"]
    re_count = tweet.retweet_count
    create_at = tweet.created_at
    try:
        text = tweet.retweeted_status.full_text
        fav_count = tweet.retweeted_status.favorite_count
    except:
        text = tweet.full_text
        fav_count = tweet.favorite_count
    new_column = pd.Series([create_at,text,hashtag,re_count,fav_count], index=df.columns)
    df = df.append(new_column,ignore_index=True)

df.to_excel("twitterCrawler7.xlsx")

In [64]:
df = pd.read_excel("twitterCrawler7.xlsx")

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

In [47]:
df["date"] = df['create_at'].dt.date

In [ ]:
df.groupby('date').sum()[['retweet_count']]

In [ ]:
df.drop_duplicates("text").sort_values(by=['retweet_count'], ascending=False).head(10)[['text','retweet_count']]

In [ ]:
df.drop_duplicates("text").sort_values(by=['favourite_count'], ascending=False).head(10)[['text','favourite_count']]

In [ ]:
def slash_tokenize(d):  
    result = d.split("/")
    result = list(filter(None, result))
    return result

hastag_data = df["hashtag"].dropna()
vectorizer = CountVectorizer(tokenizer=slash_tokenize)
transformed_data = vectorizer.fit_transform(hastag_data)
hash_tag_cnt_df= pd.DataFrame(columns = ['word', 'count']) 
hash_tag_cnt_df['word'] = vectorizer.get_feature_names()
hash_tag_cnt_df['count'] = np.ravel(transformed_data.sum(axis=0))
hash_tag_cnt_df.sort_values(by=['count'], ascending=False).head(10)

In [ ]:
def cleanText(text):
    text = str(text)
    text = text.replace('\n','')
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    text = text.replace('"',"")
    text = text.replace("!","")
    text = text.replace("​","")
    text = text.replace("-","")
    text_split = text.split(" ")
    text = " ".join([c for c in text_split if "#" not in c])
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)   
    stop_word = list(thai_stopwords())
    sentence = word_tokenize(text)
    result = [word for word in sentence if word not in stop_word and " " not in word]
    return " /".join(result)

def tokenize(d):  
    result = d.split("/")
    result = list(filter(None, result))
    return result

new_text = []
for txt in df["text"]:
    new_text.append(cleanText(txt))


vectorizer = CountVectorizer(tokenizer=tokenize)
transformed_data = vectorizer.fit_transform(new_text)
keyword_df1 = pd.DataFrame(columns = ['word', 'count'])
keyword_df1['word'] = vectorizer.get_feature_names()
keyword_df1['count'] = np.ravel(transformed_data.sum(axis=0))   
keyword_df1.sort_values(by=['count'], ascending=False).head(10)

In [ ]:
df.groupby('date').sum()[['retweet_count']].plot(kind="bar",figsize=(12, 6))